In [2]:
"""
PHASE 2: STREAMLIT ANALYTICS DASHBOARD
=======================================
A comprehensive movie analytics platform with business insights.

SETUP INSTRUCTIONS:
1. Install dependencies:
   pip install streamlit pandas numpy matplotlib seaborn plotly

2. Run the app:
   streamlit run app_phase2.py

3. Open in browser:
   http://localhost:8501

"""

'\nPHASE 2: STREAMLIT ANALYTICS DASHBOARD\n=======================================\nA comprehensive movie analytics platform with business insights.\n\nSETUP INSTRUCTIONS:\n1. Install dependencies:\n   pip install streamlit pandas numpy matplotlib seaborn plotly\n\n2. Run the app:\n   streamlit run app_phase2.py\n\n3. Open in browser:\n   http://localhost:8501\n\n'

In [4]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [6]:
# ============================================================================
# PAGE CONFIG (Must be first Streamlit command!)
# ============================================================================
st.set_page_config(
    page_title="MovieLens Analytics Dashboard",
    page_icon="🎬",
    layout="wide",  # wide = full width, centered = narrow
    initial_sidebar_state="expanded"  # Show sidebar by default
)

In [12]:
# ============================================================================
# CACHING FUNCTION (Load data once, reuse across reruns)
# ============================================================================
@st.cache_data
def load_data():
    """
    Load cleaned data from Phase 1.
    @st.cache_data decorator tells Streamlit to cache this function's output.
    Without caching, Streamlit reruns ALL code every time user interacts.
    With caching, this runs once and result is stored in memory.
    """
    df = pd.read_parquet('ratings_sample_cleaned.parquet')
    
    # Ensure proper data types
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    
    # Recreate genres_list from genres column (pipe-separated)
    df['genres_list'] = df['genres'].str.split('|')
    
    return df

# Load data
df = load_data()

2025-12-13 16:27:32.429 No runtime found, using MemoryCacheStorageManager
2025-12-13 16:27:32.432 No runtime found, using MemoryCacheStorageManager


In [14]:
# ============================================================================
# SIDEBAR: FILTERS & NAVIGATION
# ============================================================================
"""
The sidebar is where you put filters, navigation, and settings.
Users interact with sidebar widgets, which trigger reruns of your script.
"""

st.sidebar.title("🎬 MovieLens Analytics")
st.sidebar.markdown("---")

# Navigation: Choose which dashboard to view
page = st.sidebar.radio(
    "Choose Dashboard",
    ["📊 Overview", "👥 User Behavior", "🎥 Content Analysis", "🔍 Hidden Gems"],
    help="Select which analysis dashboard to view"
)

st.sidebar.markdown("---")
st.sidebar.subheader("Filters")

# Year range slider
year_range = st.sidebar.slider(
    "Select Rating Year Range",
    min_value=df['year'].min(),
    max_value=df['year'].max(),
    value=(df['year'].min(), df['year'].max()),
    help="Filter data by year of rating"
)

# Genre multi-select
available_genres = set()
for genres_list in df['genres_list']:
    if isinstance(genres_list, list):
        available_genres.update(genres_list)
available_genres = sorted(list(available_genres))

selected_genres = st.sidebar.multiselect(
    "Filter by Genre",
    available_genres,
    default=None,  # Start with no selection
    help="Leave empty to see all genres"
)

# Rating threshold
rating_threshold = st.sidebar.slider(
    "Minimum Rating Filter",
    min_value=0.5,
    max_value=5.0,
    value=2.0,
    step=0.5,
    help="Only show movies rated at least this high"
)

st.sidebar.markdown("---")

# Show data summary in sidebar
st.sidebar.subheader("📈 Data Summary")
filtered_df = df[
    (df['year'] >= year_range[0]) & 
    (df['year'] <= year_range[1]) &
    (df['rating'] >= rating_threshold)
]

st.sidebar.metric("Total Ratings", f"{len(filtered_df):,}")
st.sidebar.metric("Unique Users", f"{filtered_df['userId'].nunique():,}")
st.sidebar.metric("Unique Movies", f"{filtered_df['movieId'].nunique():,}")


DeltaGenerator(_root_container=1, _parent=DeltaGenerator())

In [16]:
# ============================================================================
# APPLY FILTERS FUNCTION (Reusable)
# ============================================================================
def apply_filters(data):
    """
    Apply sidebar filters to dataframe.
    Returns: Filtered dataframe
    """
    # Year filter
    filtered = data[
        (data['year'] >= year_range[0]) & 
        (data['year'] <= year_range[1]) &
        (data['rating'] >= rating_threshold)
    ].copy()
    
    # Genre filter (only if genres selected)
    if selected_genres:
        genre_mask = filtered['genres_list'].apply(
            lambda x: isinstance(x, list) and any(g in x for g in selected_genres)
        )
        filtered = filtered[genre_mask]
    
    return filtered

filtered_df = apply_filters(df)

In [25]:
# ============================================================================
# PAGE 1: OVERVIEW DASHBOARD
# ============================================================================
if page == "📊 Overview":
    st.title("📊 MovieLens Analytics Overview")
    st.markdown("High-level metrics and trends across the dataset")
    
    # KPI Row (Top metrics)
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric(
            label="Avg Rating",
            value=f"{filtered_df['rating'].mean():.2f}",
            delta=f"{filtered_df['rating'].mean() - 3.5:.2f} vs baseline",
            help="Overall average rating in selected filters"
        )
    
    with col2:
        st.metric(
            label="Median Rating",
            value=f"{filtered_df['rating'].median():.1f}",
            help="Middle value (50th percentile)"
        )
    
    with col3:
        st.metric(
            label="Rating Std Dev",
            value=f"{filtered_df['rating'].std():.2f}",
            help="Spread of ratings (lower = more consistent)"
        )
    
    with col4:
        st.metric(
            label="Data Points",
            value=f"{len(filtered_df):,}",
            help="Number of ratings in current filter"
        )
    
    st.markdown("---")
    
    # Two-column layout for charts
    col1, col2 = st.columns(2)
    
    # Chart 1: Rating Distribution (Histogram)
    with col1:
        st.subheader("Rating Distribution")
        
        rating_counts = filtered_df['rating'].value_counts().sort_index()
        
        fig = px.bar(
            x=rating_counts.index,
            y=rating_counts.values,
            labels={'x': 'Rating', 'y': 'Count'},
            title="How ratings are distributed",
            color_continuous_scale='Blues'
        )
        fig.update_layout(
            height=400,
            showlegend=False,
            hovermode='x unified'
        )
        st.plotly_chart(fig, use_container_width=True)
    
    # Chart 2: Ratings Over Time (Line)
    with col2:
        st.subheader("Rating Trends Over Time")
        
        ratings_by_year = filtered_df.groupby('year').agg({
            'rating': ['mean', 'count']
        }).reset_index()
        ratings_by_year.columns = ['year', 'avg_rating', 'count']
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=ratings_by_year['year'],
            y=ratings_by_year['avg_rating'],
            mode='lines+markers',
            name='Avg Rating',
            line=dict(color='#1f77b4', width=3),
            marker=dict(size=8)
        ))
        
        fig.update_layout(
            title="Are ratings increasing or decreasing?",
            xaxis_title="Year",
            yaxis_title="Average Rating",
            height=400,
            hovermode='x unified',
            template='plotly_white'
        )
        st.plotly_chart(fig, use_container_width=True)
    
    st.markdown("---")
    
    # Genre Performance Analysis (wide chart)
    st.subheader("Genre Performance Analysis")
    
    # Extract genres and ratings
    all_genres = []
    genre_ratings = []
    for genres_list, rating in zip(filtered_df['genres_list'], filtered_df['rating']):
        if isinstance(genres_list, list):
            for g in genres_list:
                all_genres.append(g)
                genre_ratings.append(rating)
    
    genre_df = pd.DataFrame({'genre': all_genres, 'rating': genre_ratings})
    genre_stats = genre_df.groupby('genre').agg({
        'rating': ['mean', 'count', 'std']
    }).round(2)
    genre_stats.columns = ['avg_rating', 'count', 'std_dev']
    genre_stats = genre_stats.sort_values('avg_rating', ascending=False)
    
    # Plotly bar chart with hover info
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=genre_stats.index,
        y=genre_stats['avg_rating'],
        marker=dict(
            color=genre_stats['avg_rating'],
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="Avg Rating")
        ),
        text=genre_stats['avg_rating'].round(2),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Avg Rating: %{y:.2f}<br>Count: ' + 
                      genre_stats['count'].astype(str) + '<extra></extra>'
    ))
    
    fig.update_layout(
        title="Which genres are rated highest?",
        xaxis_title="Genre",
        yaxis_title="Average Rating",
        height=400,
        hovermode='x unified',
        template='plotly_white'
    )
    st.plotly_chart(fig, use_container_width=True)
    
    # Expander: Show raw genre stats
    with st.expander("📋 View Genre Statistics Table"):
        st.dataframe(genre_stats.reset_index(), use_container_width=True)
# ============================================================================
# PAGE 2: USER BEHAVIOR ANALYSIS
# ============================================================================
elif page == "👥 User Behavior":
    st.title("👥 User Behavior Analysis")
    st.markdown("Understanding how different users rate movies")
    
    # User statistics
    user_stats = filtered_df.groupby('userId').agg({
        'rating': ['count', 'mean', 'std', 'min', 'max']
    }).reset_index()
    user_stats.columns = ['userId', 'num_ratings', 'avg_rating', 'rating_std', 'min_rating', 'max_rating']
    
    # KPI cards
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        harsh_count = (user_stats['avg_rating'] < 3).sum()
        st.metric(
            "Harsh Raters (<3.0)",
            harsh_count,
            f"{harsh_count/len(user_stats)*100:.1f}% of users"
        )
    
    with col2:
        moderate_count = ((user_stats['avg_rating'] >= 3) & (user_stats['avg_rating'] < 3.5)).sum()
        st.metric(
            "Moderate Raters (3-3.5)",
            moderate_count,
            f"{moderate_count/len(user_stats)*100:.1f}% of users"
        )
    
    with col3:
        generous_count = (user_stats['avg_rating'] >= 3.5).sum()
        st.metric(
            "Generous Raters (≥3.5)",
            generous_count,
            f"{generous_count/len(user_stats)*100:.1f}% of users"
        )
    
    with col4:
        highly_active = (user_stats['num_ratings'] > 50).sum()
        st.metric(
            "Highly Active Users",
            highly_active,
            f"{highly_active/len(user_stats)*100:.1f}% of users"
        )
    
    st.markdown("---")
    
    col1, col2 = st.columns(2)
    
    # Chart 1: Distribution of user average ratings
    with col1:
        st.subheader("User Rating Behavior Distribution")
        
        fig = px.histogram(
            user_stats,
            x='avg_rating',
            nbins=50,
            labels={'avg_rating': 'Average Rating per User', 'count': 'Number of Users'},
            title="Are users harsh or generous raters?",
            color_discrete_sequence=['#FF6B6B']
        )
        
        # Add mean line
        mean_rating = user_stats['avg_rating'].mean()
        fig.add_vline(
            x=mean_rating,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Mean: {mean_rating:.2f}",
            annotation_position="top right"
        )
        
        fig.update_layout(
            height=400,
            hovermode='x unified',
            template='plotly_white'
        )
        st.plotly_chart(fig, use_container_width=True)
    
    # Chart 2: Activity distribution (how many ratings per user)
    with col2:
        st.subheader("User Activity Distribution")
        
        fig = px.histogram(
            user_stats,
            x='num_ratings',
            nbins=50,
            labels={'num_ratings': 'Number of Ratings', 'count': 'Number of Users'},
            title="How active are users? (ratings per user)",
            color_discrete_sequence=['#4ECDC4'],
            log_y=True  # Log scale because distribution is very skewed
        )
        
        median_ratings = user_stats['num_ratings'].median()
        fig.add_vline(
            x=median_ratings,
            line_dash="dash",
            line_color="orange",
            annotation_text=f"Median: {median_ratings:.0f}",
            annotation_position="top right"
        )
        
        fig.update_layout(
            height=400,
            hovermode='x unified',
            template='plotly_white'
        )
        st.plotly_chart(fig, use_container_width=True)
    
    st.markdown("---")
    
    # Scatter: Activity vs Rating Consistency
    st.subheader("User Activity vs Rating Consistency")
    
    fig = px.scatter(
        user_stats,
        x='num_ratings',
        y='rating_std',
        hover_data=['avg_rating', 'min_rating', 'max_rating'],
        title="Do active users rate more consistently?",
        labels={
            'num_ratings': 'Number of Ratings',
            'rating_std': 'Rating Standard Deviation (Consistency)'
        },
        opacity=0.6,
        color='avg_rating',
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(
        height=450,
        hovermode='closest',
        template='plotly_white'
    )
    st.plotly_chart(fig, use_container_width=True)
    
    st.markdown("---")
    
    # User segmentation table
    st.subheader("User Segments Summary")
    
    segments = {
        'Harsh & Inactive': (user_stats['avg_rating'] < 3) & (user_stats['num_ratings'] < 10),
        'Harsh & Active': (user_stats['avg_rating'] < 3) & (user_stats['num_ratings'] >= 10),
        'Moderate & Inactive': ((user_stats['avg_rating'] >= 3) & (user_stats['avg_rating'] < 3.5)) & (user_stats['num_ratings'] < 10),
        'Moderate & Active': ((user_stats['avg_rating'] >= 3) & (user_stats['avg_rating'] < 3.5)) & (user_stats['num_ratings'] >= 10),
        'Generous & Inactive': (user_stats['avg_rating'] >= 3.5) & (user_stats['num_ratings'] < 10),
        'Generous & Active': (user_stats['avg_rating'] >= 3.5) & (user_stats['num_ratings'] >= 10),
    }
    
    segment_data = []
    for segment_name, mask in segments.items():
        count = mask.sum()
        pct = count / len(user_stats) * 100
        segment_data.append({
            'Segment': segment_name,
            'User Count': count,
            'Percentage': f'{pct:.1f}%'
        })
    
    segment_df = pd.DataFrame(segment_data)
    st.dataframe(segment_df, use_container_width=True, hide_index=True)

# ============================================================================
# PAGE 3: CONTENT ANALYSIS
# ============================================================================
elif page == "🎥 Content Analysis":
    st.title("🎥 Content Analysis")
    st.markdown("Deep dive into movie characteristics and ratings")
    
    # Movie statistics
    movie_stats = filtered_df.groupby('movieId').agg({
        'rating': ['count', 'mean'],
        'title': 'first',
        'release_year': 'first'
    }).reset_index()
    movie_stats.columns = ['movieId', 'num_ratings', 'avg_rating', 'title', 'release_year']
    
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric(
            "Total Movies",
            movie_stats['movieId'].nunique(),
            help="Number of unique movies rated"
        )
    
    with col2:
        st.metric(
            "Avg Ratings/Movie",
            f"{movie_stats['num_ratings'].mean():.0f}",
            help="Average number of ratings per movie"
        )
    
    with col3:
        st.metric(
            "Median Ratings/Movie",
            f"{movie_stats['num_ratings'].median():.0f}",
            help="Median shows skew (some blockbusters, many niche)"
        )
    
    with col4:
        st.metric(
            "Most Rated Movie",
            movie_stats['num_ratings'].max(),
            help="Maximum ratings any single movie has"
        )
    
    st.markdown("---")
    
    col1, col2 = st.columns(2)
    
    # Chart 1: Release year impact
    with col1:
        st.subheader("Release Year Impact on Ratings")
        
        year_stats = filtered_df.dropna(subset=['release_year']).groupby('release_year').agg({
            'rating': ['mean', 'count']
        }).reset_index()
        year_stats.columns = ['release_year', 'avg_rating', 'count']
        year_stats = year_stats[year_stats['count'] > 5]  # Filter noise
        
        fig = px.scatter(
            year_stats,
            x='release_year',
            y='avg_rating',
            size='count',
            title="Do older/newer movies get different ratings?",
            labels={
                'release_year': 'Movie Release Year',
                'avg_rating': 'Average Rating',
                'count': 'Number of Ratings'
            },
            color='avg_rating',
            color_continuous_scale='RdYlGn',
            hover_data=['count']
        )
        
        fig.update_layout(
            height=400,
            hovermode='closest',
            template='plotly_white'
        )
        st.plotly_chart(fig, use_container_width=True)
    
    # Chart 2: Popularity distribution (long-tail)
    with col2:
        st.subheader("Movie Popularity Distribution (Long-Tail)")
        
        fig = px.histogram(
            movie_stats,
            x='num_ratings',
            nbins=50,
            title="Few blockbusters, many niche movies?",
            labels={'num_ratings': 'Ratings per Movie', 'count': 'Number of Movies'},
            color_discrete_sequence=['#95E1D3'],
            log_y=True
        )
        
        fig.update_xaxes(type='log')
        
        fig.update_layout(
            height=400,
            hovermode='x unified',
            template='plotly_white'
        )
        st.plotly_chart(fig, use_container_width=True)
    
    st.markdown("---")
    
    # Top and Bottom rated movies
    st.subheader("Top-Rated vs Lowest-Rated Movies")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.markdown("**🏆 Top 10 Highest-Rated (min 10 ratings)**")
        top_movies = movie_stats[movie_stats['num_ratings'] >= 10].nlargest(10, 'avg_rating')
        
        for idx, (_, row) in enumerate(top_movies.iterrows(), 1):
            st.write(
                f"{idx}. **{row['title']}** ({row['release_year']:.0f})\n"
                f"   Rating: ⭐ {row['avg_rating']:.2f} ({int(row['num_ratings'])} ratings)"
            )
    
    with col2:
        st.markdown("**👎 Lowest 10 Rated (min 10 ratings)**")
        bottom_movies = movie_stats[movie_stats['num_ratings'] >= 10].nsmallest(10, 'avg_rating')
        
        for idx, (_, row) in enumerate(bottom_movies.iterrows(), 1):
            st.write(
                f"{idx}. **{row['title']}** ({row['release_year']:.0f})\n"
                f"   Rating: ⭐ {row['avg_rating']:.2f} ({int(row['num_ratings'])} ratings)"
            )

# ============================================================================
# PAGE 4: HIDDEN GEMS FINDER
# ============================================================================
elif page == "🔍 Hidden Gems":
    st.title("🔍 Hidden Gems Finder")
    st.markdown("Discover underrated movies with high ratings but low visibility")
    
    # Define hidden gems: high rating + few ratings
    movie_stats = filtered_df.groupby('movieId').agg({
        'rating': ['count', 'mean'],
        'title': 'first',
        'release_year': 'first'
    }).reset_index()
    movie_stats.columns = ['movieId', 'num_ratings', 'avg_rating', 'title', 'release_year']
    
    st.subheader("Criteria for Hidden Gems")
    col1, col2, col3 = st.columns(3)
    
    with col1:
        min_rating = st.slider(
            "Minimum Rating",
            min_value=3.0,
            max_value=5.0,
            value=4.0,
            step=0.1,
            key="hidden_gems_min_rating"
        )
    
    with col2:
        max_visibility = st.slider(
            "Maximum Ratings (Low Visibility)",
            min_value=1,
            max_value=100,
            value=20,
            key="hidden_gems_max_visibility"
        )
    
    with col3:
        min_gem_ratings = st.slider(
            "Minimum Gem Ratings",
            min_value=1,
            max_value=10,
            value=3,
            key="hidden_gems_min_ratings"
        )
    
    # Filter hidden gems
    hidden_gems = movie_stats[
        (movie_stats['avg_rating'] >= min_rating) &
        (movie_stats['num_ratings'] <= max_visibility) &
        (movie_stats['num_ratings'] >= min_gem_ratings)
    ].sort_values('avg_rating', ascending=False)
    
    # Display summary
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.metric("Hidden Gems Found", len(hidden_gems))
    
    with col2:
        st.metric("Avg Hidden Gem Rating", f"{hidden_gems['avg_rating'].mean():.2f}")
    
    with col3:
        st.metric("Avg Visibility (Ratings)", f"{hidden_gems['num_ratings'].mean():.0f}")
    
    st.markdown("---")
    
    # Visualization: Hidden gems scatter
    st.subheader("Hidden Gems Scatter Plot")
    
    fig = px.scatter(
        movie_stats,
        x='num_ratings',
        y='avg_rating',
        hover_data=['title', 'release_year'],
        title="Find the sweet spot: High Rating + Low Visibility",
        labels={
            'num_ratings': 'Number of Ratings (Visibility)',
            'avg_rating': 'Average Rating'
        },
        opacity=0.5,
        color='release_year',
        color_continuous_scale='Viridis'
    )
    
    # Highlight hidden gems region
    fig.add_shape(
        type="rect",
        x0=0, x1=max_visibility,
        y0=min_rating, y1=5.0,
        fillcolor="gold",
        opacity=0.15,
        line=dict(color="gold", width=2, dash="dash"),
        name="Hidden Gems Zone"
    )
    
    fig.update_layout(
        height=500,
        hovermode='closest',
        template='plotly_white'
    )
    st.plotly_chart(fig, use_container_width=True)
    
    st.markdown("---")
    
    # Hidden gems list
    st.subheader(f"📚 {len(hidden_gems)} Hidden Gems Discovered")
    
    if len(hidden_gems) > 0:
        # Paginate if too many
        page_size = 10
        total_pages = (len(hidden_gems) + page_size - 1) // page_size
        
        page_num = st.selectbox(
            "Page",
            range(1, total_pages + 1),
            help=f"Total {len(hidden_gems)} hidden gems across {total_pages} pages"
        )
        
        start_idx = (page_num - 1) * page_size
        end_idx = start_idx + page_size
        
        for idx, (_, row) in enumerate(hidden_gems.iloc[start_idx:end_idx].iterrows(), start_idx + 1):
            col1, col2, col3 = st.columns([3, 1, 1])
            
            with col1:
                st.write(f"**{idx}. {row['title']}** ({row['release_year']:.0f})")
            
            with col2:
                st.metric("Rating", f"{row['avg_rating']:.2f}", label_visibility="collapsed")
            
            with col3:
                st.metric("Ratings", f"{int(row['num_ratings'])}", label_visibility="collapsed")
    else:
        st.warning("No hidden gems found with current criteria. Try adjusting the filters!")
    

In [27]:
# ============================================================================
# FOOTER
# ============================================================================
st.markdown("---")
st.markdown(
    """
    **MovieLens Analytics Dashboard** | Phase 2 - Business Insights
    
    📊 Filters active in sidebar | 💾 Data from Phase 1 EDA | 🚀 Ready for Phase 3 ML Models
    """
)

DeltaGenerator()